In [1]:
%reload_ext autoreload
%autoreload 2
import sys
sys.path.append('../')
import gensim
import numpy as np
import json
from brand.debiasing import EmbeddingDebias
from brand.utils import get_embedding_mat

# load word vector model
pretrained_embedding_file = '../data/GoogleNews-vectors-negative300.bin.gz'
kv = gensim.models.KeyedVectors.load_word2vec_format(pretrained_embedding_file, binary=True, limit=500000)

In [10]:
cos_sim_1 = kv['Didi'].dot(kv['Uber']) / (np.linalg.norm(kv['Didi']) * np.linalg.norm(kv['Uber']))
cos_sim_2 = kv['Didi'].dot(kv['Alibaba']) / (np.linalg.norm(kv['Didi']) * np.linalg.norm(kv['Alibaba']))

print('cos similarity between Didi and Uber:', cos_sim_1)
print('cos similarity between Didi and Alibaba:', cos_sim_2)

cos similarity between Didi and Uber: 0.1372267
cos similarity between Didi and Alibaba: 0.10909378


In [11]:
# debiasing country
# load gender specific words
country_cfg_file = '../vocab/country_us_china.json'
with open(country_cfg_file, 'r') as f:
    cfg = json.load(f)
    country_ds = cfg['definite_sets']

country_dmat = []
for _, words in enumerate(country_ds):
    mat = get_embedding_mat(words, kv)
    country_dmat.append(np.asarray(mat))

method = 'Hard'
k = 1
country_debias = EmbeddingDebias(country_dmat, embedding=kv, k=k, method=method)

words = ['Didi', 'Uber', 'Alibaba']
_, embedding_debiased, _ = country_debias.debiasing(words=words, eq_sets=country_ds)

In [12]:
# check cosine similarity after debiasing country
cos_sim_1 = embedding_debiased[0].dot(embedding_debiased[1]) / (np.linalg.norm(embedding_debiased[0]) * np.linalg.norm(embedding_debiased[1]))
cos_sim_2 = embedding_debiased[0].dot(embedding_debiased[2]) / (np.linalg.norm(embedding_debiased[0]) * np.linalg.norm(embedding_debiased[2]))

print('cos similarity between Didi and Uber:', cos_sim_1)
print('cos similarity between Didi and Alibaba:', cos_sim_2)

cos similarity between Didi and Uber: 0.13842283
cos similarity between Didi and Alibaba: 0.085968584


In [14]:
brand1 = 'Baidu'
brand2 = 'Google'
brand3 = 'Didi'
cos_sim_1 = kv[brand1].dot(kv[brand2]) / (np.linalg.norm(kv[brand1]) * np.linalg.norm(kv[brand2]))
cos_sim_2 = kv[brand1].dot(kv[brand3]) / (np.linalg.norm(kv[brand1]) * np.linalg.norm(kv[brand3]))

print('cos similarity between {} and {}:'.format(brand1, brand2), cos_sim_1)
print('cos similarity between {} and {}:'.format(brand1, brand3), cos_sim_2)

cos similarity between Baidu and Google: 0.69934654
cos similarity between Baidu and Didi: 0.08659812


In [ ]:
words = [brand1, brand2, brand3]
_, embedding_debiased, _ = country_debias.debiasing(words=words, eq_sets=country_ds)
cos_sim_1 = embedding_debiased[0].dot(embedding_debiased[1]) / (np.linalg.norm(embedding_debiased[0]) * np.linalg.norm(embedding_debiased[1]))
cos_sim_2 = embedding_debiased[0].dot(embedding_debiased[2]) / (np.linalg.norm(embedding_debiased[0]) * np.linalg.norm(embedding_debiased[2]))

print('cos similarity between {} and {}:'.format(brand1, brand2), cos_sim_1)
print('cos similarity between {} and {}:'.format(brand1, brand3), cos_sim_2)